In [1]:
!gdown 1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
!gdown 17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq

Downloading...
From: https://drive.google.com/uc?id=1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource1-286218-17243904498090.xlsx
100%|██████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 18.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource2-286218-17243904776879.xlsx
100%|██████████████████████████████████████| 12.7k/12.7k [00:00<00:00, 18.0MB/s]


In [2]:
import pandas as pd
from pandas import DataFrame

orderSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='orderSource1')
productSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='productSource1')
stateLookupSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='StateLookup')

orderSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='orderSource2')
productSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='productSource2')